# THE Asia Ranking
- https://www.timeshighereducation.com/world-university-rankings/2023/regional-ranking
- https://www.timeshighereducation.com/world-university-rankings/2024/regional-ranking
- https://www.timeshighereducation.com/world-university-rankings/2025/regional-ranking
- part : #!/length/-1/sort_by/rank/sort_order/asc/cols/scores

In [23]:
urls = ["https://www.timeshighereducation.com/world-university-rankings/2023/regional-ranking/", 
"https://www.timeshighereducation.com/world-university-rankings/2024/regional-ranking/", "https://www.timeshighereducation.com/world-university-rankings/2025/regional-ranking"]
part1 = '#!/length/-1/sort_by/rank/sort_order/asc/cols/scores/'
urls[0] + part1

'https://www.timeshighereducation.com/world-university-rankings/2023/regional-ranking/#!/length/-1/sort_by/rank/sort_order/asc/cols/scores/'

In [20]:
import requests
import urllib
from bs4 import BeautifulSoup
response = requests.get('http://google.com')
assert response.status_code < 400

In [18]:
#functino check if url exists
def get_page_title(url):
    try:
        response = requests.get(url, timeout=5)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            return soup.title.string.strip() if soup.title else "No title found"
        else:
            return f"Status code: {response.status_code}"
    except requests.RequestException as e:
        return f"Error: {e}"

In [25]:
print(get_page_title("https://www.google.com"))
print(get_page_title(urls[0] + part1))

Google
Status code: 403


In [ ]:
#install libraries if required

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import pandas as pd
import numpy as np
import re

In [3]:
pd.set_option('max_colwidth',None)

In [4]:
# Setup Chrome options
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--disable-notifications")

In [5]:
# Download location
prefs = { "download.prompt_for_download": False, "download.default_directory": "/Users/du/dup/auData/download"}
options.add_experimental_option("prefs", prefs)

In [6]:
cp2 = '/opt/homebrew/bin/chromedriver'  # Replace with the actual path to your chromedriver

In [7]:
#start driver
cService = webdriver.ChromeService(executable_path=cp2)
driver = webdriver.Chrome(service = cService)
#from webdriver_manager.chrome import ChromeDriverManager
#service = Service(ChromeDriverManager().install())
#driver = webdriver.Chrome(service=service)
#driver = webdriver.Chrome(ChromeDriverManager().install())
#print(driver.capabilities)

In [8]:
cService = webdriver.ChromeService(executable_path=cp2)
driver = webdriver.Chrome(service = cService)
url = urls[0] + part1
driver.get(url)
wait = WebDriverWait(driver, 5)
# Maximize window and wait for page to load
driver.maximize_window()
time.sleep(5)  
driver.find_element(By.ID,'CybotCookiebotDialogBodyButtonDecline').click()

In [9]:
urls

['https://www.timeshighereducation.com/world-university-rankings/2023/regional-ranking/',
 'https://www.timeshighereducation.com/world-university-rankings/2024/regional-ranking/',
 'https://www.timeshighereducation.com/world-university-rankings/2025/regional-ranking']

Go to: System Settings > Privacy & Security
	2.	Scroll down to Security section
	3.	You’ll see: “chromedriver was blocked from use because it is not from an identified developer”
	4.	Click “Allow Anyway”
xattr -d com.apple.quarantine chromedriver

In [10]:
# Ranking Tab
#once cookies has been stopped run this code again
ranking_headers = {}
for base_url in urls:
    full_url = base_url + part1
    driver.get(full_url)
    year = re.search(r'\d{4}', driver.title).group()
    print(year)
    wait = WebDriverWait(driver, 2)
    ranking_label = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "label[for='scores']")))
    ranking_label.click()
    time.sleep(2)
    
    table = driver.find_element(By.ID, "datatable-1")
    headers = table.find_elements(By.TAG_NAME, "th")
    ranking_headers[year] = [h.text.replace('\n', ' ').strip() for h in headers]
    #print(ranking_headers)

df_headers = pd.DataFrame([ {'year': year, 'variables': ', '.join(cols)} for year, cols in ranking_headers.items()])
df_headers
#ranking_headers

2023
2024
2025


,year,variables
0,2023,"Rank, Name Country/Region, Overall, Citations, Industry Income, International Outlook, Research, Teaching"
1,2024,"Rank, Name Country/Region, Overall, Research Quality, Industry, International Outlook, Research Environment, Teaching"
2,2025,"Rank, Name Country/Region, Overall, Research Quality, Industry, International Outlook, Research Environment, Teaching"


In [11]:
df_headers

,year,variables
0,2023,"Rank, Name Country/Region, Overall, Citations, Industry Income, International Outlook, Research, Teaching"
1,2024,"Rank, Name Country/Region, Overall, Research Quality, Industry, International Outlook, Research Environment, Teaching"
2,2025,"Rank, Name Country/Region, Overall, Research Quality, Industry, International Outlook, Research Environment, Teaching"


In [ ]:
ranking_headers

In [31]:
driver.refresh()

In [55]:
full_url = urls[2] + part1
driver.get(full_url)
time.sleep(2) 
year = re.search(r'\d{4}', driver.title).group()
print(f'Year {year} Title - {driver.title} ')
ranking_table = driver.find_element(By.ID, "datatable-1")
rows = ranking_table.find_elements(By.TAG_NAME, "tr")

ranking_data = []
for row in rows[1:]:
    cols = row.find_elements(By.TAG_NAME, "td")
    ranking_data.append([col.text for col in cols])

Year 2025 Title - Asia University Rankings 2025 | Times Higher Education (THE) 


In [56]:
pd.DataFrame(ranking_data[0:1])

,0,1,2,3,4,5,6,7
0,1,Tsinghua University\nChina,93.0,94.8,100.0,49.8,98.4,94.9


In [57]:
columns1 = ['rank','university','overall','citations','industryIncome','intlOutlook', 'research','teaching']
df1 = pd.DataFrame(ranking_data, columns=columns1)
df1['theYear'] ='Asia_'+ year + '_score'
df1.shape

(1213, 9)

In [58]:
df1.head()

,rank,university,overall,citations,industryIncome,intlOutlook,research,teaching,theYear
0,1,Tsinghua University\nChina,93.0,94.8,100.0,49.8,98.4,94.9,Asia_2025_score
1,2,Peking University\nChina,92.9,90.4,99.9,70.1,97.9,94.3,Asia_2025_score
2,3,National University of Singapore\nSingapore\nExplore,91.3,97.2,100.0,91.9,94.4,76.6,Asia_2025_score
3,4,"Nanyang Technological University, Singapore\nSingapore\nExplore",85.4,96.7,99.9,94.1,82.0,66.7,Asia_2025_score
4,5,The University of Tokyo\nJapan,84.8,72.9,100.0,50.4,93.4,93.7,Asia_2025_score


In [59]:
# Key Statistics Tabs
# Click the label for "Key Statistics" tab
key_stats_label = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "label[for='stats']")))
key_stats_label.click()
time.sleep(5)

In [60]:
table = driver.find_element(By.ID, "datatable-1")
rows = table.find_elements(By.TAG_NAME, "tr")

key_stats_data = []
for row in rows[1:]:
    cols = row.find_elements(By.TAG_NAME, "td")
    key_stats_data.append([col.text for col in cols])
key_stats_data[0:1]

[['1', 'Tsinghua University\nChina', '38,289', '11.0', '9%', 'n/a']]

In [61]:
columns2 = ['rank','university','students','stuStaffRatio','intlStudents','FMratio']
df2 = pd.DataFrame(key_stats_data, columns=columns2)
df2['theYear'] = 'Asia_'+ year + '_stats'
df2.head(2)

,rank,university,students,stuStaffRatio,intlStudents,FMratio,theYear
0,1,Tsinghua University\nChina,"38,289",11.0,9%,n/a,Asia_2025_stats
1,2,Peking University\nChina,"34,981",11.5,15%,n/a,Asia_2025_stats


In [62]:
df2.head(5)

,rank,university,students,stuStaffRatio,intlStudents,FMratio,theYear
0,1,Tsinghua University\nChina,"38,289",11.0,9%,n/a,Asia_2025_stats
1,2,Peking University\nChina,"34,981",11.5,15%,n/a,Asia_2025_stats
2,3,National University of Singapore\nSingapore\nExplore,"34,388",21.0,26%,49 : 51,Asia_2025_stats
3,4,"Nanyang Technological University, Singapore\nSingapore\nExplore","26,044",17.1,29%,48 : 52,Asia_2025_stats
4,5,The University of Tokyo\nJapan,"26,438",10.4,17%,n/a,Asia_2025_stats


In [63]:
df2[['university', 'location']] = df2['university'].str.split('\n', n=1, expand=True)
df2.head(2)

,rank,university,students,stuStaffRatio,intlStudents,FMratio,theYear,location
0,1,Tsinghua University,"38,289",11.0,9%,n/a,Asia_2025_stats,China
1,2,Peking University,"34,981",11.5,15%,n/a,Asia_2025_stats,China


In [64]:
dfs = {'stats': df1, 'keystats': df2}

# Save to Excel file
with pd.ExcelWriter('/Users/du/dup/auData/ranking/the/asia_'+year+'.xlsx', engine='xlsxwriter') as writer:
    for sheet_name, df in dfs.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

In [ ]:
df2 = pd.DataFrame(key_stats_data, columns=columns2)
df2['theYear'] = 'Asia_2023'
df2[['university', 'location']] = df2['university'].str.split('\n', n=1, expand=True)
df2['intlStudents'] = df2['intlStudents'].str.replace('%', '', regex=False).str.strip()
df2['intlStudents'] = pd.to_numeric(df2['intlStudents'], errors='coerce')

df2['students'] = df2['students'].str.replace(',', '', regex=False)
df2['students'] = pd.to_numeric(df2['students'], errors='coerce')

df2['FMratio'] = df2['FMratio'].str.split(':').str[0].str.replace(' ', '')
df2['FMratio'] = pd.to_numeric(df2['FMratio'], errors='coerce')
df2['location'] = df2['location'].str.replace('\nExplore', '', regex=False)

In [ ]:
df2.head()

In [ ]:
# Now I want it to scroll through multiple pages and combine in to one excel sheet
urls = ["https://www.timeshighereducation.com/world-university-rankings/2023/regional-ranking/", 
"https://www.timeshighereducation.com/world-university-rankings/2024/regional-ranking/", "https://www.timeshighereducation.com/world-university-rankings/2025/regional-ranking"]
part1 = '#!/length/-1/sort_by/rank/sort_order/asc/cols/scores/'

In [ ]:
#check if headings are same
# Initialize
cService = webdriver.ChromeService(executable_path=cp2)
driver = webdriver.Chrome(service = cService)
ranking_headers = {}
key_stats_headers = {}

for url in urls:
    full_url = url + part1
    driver.get(full_url)
    time.sleep(5)
    
    # Handle cookie reject
    try:
        reject_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.ID, "CybotCookiebotDialogBodyButtonDecline")) )
        reject_button.click()
        time.sleep(2)
    except TimeoutException:
        print(f"No cookie dialog for {full_url}")
    #extract year
    year = re.search(r'\d{4}', driver.title).group()

    # --- RANKING TAB HEADERS ---
    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "label[for='scores']"))).click()
    time.sleep(3)
    table = driver.find_element(By.ID, "datatable-1")
    headers = table.find_elements(By.TAG_NAME, "th")
    ranking_headers[year] = [h.text.replace('\n', ' ').strip() for h in headers]

    # --- KEY STATISTICS TAB HEADERS ---
    #wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "label[for='stats']"))).click()
    #ime.sleep(3)
    #table = driver.find_element(By.ID, "datatable-1")
    #headers = table.find_elements(By.TAG_NAME, "th")
    #key_stats_headers[year] = [h.text.replace('\n', ' ').strip() for h in headers]

# Print output
print("Ranking Table Headers:")
for year, cols in ranking_headers.items():
    print(f"{year}: {cols}")

#print("\nKey Statistics Table Headers:")
#for year, cols in key_stats_headers.items():
    #print(f"{year}: {cols}")

In [ ]:
all_combined =[]
for url in urls:
    full_url = url + part1
    driver.get(full_url)
    print(driver.title)
    year = re.search(r'\d{4}', driver.title).group()
    label = 'THE_Asia_' + year
    print(label)
    wait = WebDriverWait(driver, 20)
    driver.maximize_window()
    time.sleep(5)

#----RANKING
    ranking_label = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "label[for='scores']")))
    ranking_label.click()
    time.sleep(5)
#------
    ranking_table = driver.find_element(By.ID, "datatable-1")
    rows = ranking_table.find_elements(By.TAG_NAME, "tr")

    ranking_data = []
    for row in rows[1:]:
        cols = row.find_elements(By.TAG_NAME, "td")
    ranking_data.append([col.text for col in cols])
    columns1 = ['rank','university1','overall','citations','industryIncome','intlOutlook', 'research','teaching']
    df1 = pd.DataFrame(ranking_data, columns=columns1)
    df1['theYear'] = label
    df1['page1'] ='Ranking'
    df1[['university1', 'location1']] = df1['university1'].str.split('\n', n=1, expand=True)
    df1['location1'] = df1['location1'].str.replace('\nExplore', '', regex=False)
    print('DF1 done for ', label)
#---keystats-------
    key_stats_label = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "label[for='stats']")))
    key_stats_label.click()
    time.sleep(5)
#-------
    key_stats_table = driver.find_element(By.ID, "datatable-1")
    rows = key_stats_table.find_elements(By.TAG_NAME, "tr")

    key_stats_data = []
    for row in rows[1:]:
        cols = row.find_elements(By.TAG_NAME, "td")
        key_stats_data.append([col.text for col in cols])
#-------
    columns2 = ['rank','university2','students','stuStaffRatio','intlStudents','FMratio']
    df2 = pd.DataFrame(key_stats_data, columns=columns2)
    df2['theYear'] = label
    df2['page2'] ='KeyStatistics'
#--------
    df2[['university2', 'location2']] = df2['university2'].str.split('\n', n=1, expand=True)
    df2['intlStudents'] = df2['intlStudents'].str.replace('%', '', regex=False).str.strip()
    df2['intlStudents'] = pd.to_numeric(df2['intlStudents'], errors='coerce')

    df2['students'] = df2['students'].str.replace(',', '', regex=False)
    df2['students'] = pd.to_numeric(df2['students'], errors='coerce')

    df2['FMratio'] = df2['FMratio'].str.split(':').str[0].str.replace(' ', '')
    df2['FMratio'] = pd.to_numeric(df2['FMratio'], errors='coerce')
    df2['location2'] = df2['location2'].str.replace('\nExplore', '', regex=False)
    print('DF2 done for ', label)
    # --- Combine horizontally ---
    df_combined = pd.concat([df1.reset_index(drop=True), df2.reset_index(drop=True)], axis=1)
    print('DF1 & DF2 combined for ', label)
    # --- Append to final list ---
    all_combined.append(df_combined)

# --- Combine all years (row bind) ---
final_df = pd.concat(all_combined, axis=0, ignore_index=True)
#print(final_df.head())

In [ ]:
final_df.head()

In [ ]:
df2.head()

In [ ]:
# Open the THE Rankings page
url = "https://www.timeshighereducation.com/world-university-rankings/2025/regional-ranking#!/length/-1/sort_by/rank/sort_order/asc/cols/scores"
driver.get(url)
# Wait for the table to load
time.sleep(10)

# Click download button if available
try:
    download_button = driver.find_element(By.CSS_SELECTOR, "div.download a[title='Download table data']")
    download_button.click()
    print("Download started...")
except Exception as e:
    print("Download failed:", e)

# Wait a bit for download
time.sleep(10)


In [ ]:
driver.quit()

In [ ]:
driver.refresh()

In [ ]:
# Locate table by ID
table = driver.find_element("id", "datatable-1")
html = table.get_attribute("outerHTML")

# Convert to DataFrame using pandas
df = pd.read_html(html)[0]
#driver.quit()

In [ ]:
df1.head()

In [ ]:
df.to_csv("/Users/du/dup/auData/download/theasia25a.csv", index=False)

In [ ]:
# Find table
table = driver.find_element(By.ID, "datatable-1")
rows = table.find_elements(By.CSS_SELECTOR, "tbody tr")

# Extract data
data = []
for row in rows:
    cols = row.find_elements(By.TAG_NAME, "td")
    if len(cols) >= 5:
        rank = cols[0].text.strip()
        name = cols[1].find_element(By.CLASS_NAME, "ranking-institution-title").text.strip()
        country = cols[1].find_element(By.CLASS_NAME, "location").text.strip()
        fte_students = cols[2].text.strip()
        student_staff_ratio = cols[3].text.strip()
        intl_students = cols[4].text.strip()
        data.append({
            "Rank": rank,
            "Name": name,
            "Country": country,
            "FTE Students": fte_students,
            "Student-Staff Ratio": student_staff_ratio,
            "International Students": intl_students
        })

# Close browser
#driver.quit()

# Convert to DataFrame
df2 = pd.DataFrame(data)

In [ ]:
df2.head()

In [ ]:
df2.to_csv("/Users/du/dup/auData/download/theasia25b.csv", index=False)

In [ ]:
driver.quit()

## practise

In [ ]:
# Initialize
cService = webdriver.ChromeService(executable_path=cp2)
driver = webdriver.Chrome(service = cService)
full_url = urls[0] + part1
print(full_url)
driver.get(full_url)
time.sleep(5)
    
# Handle cookie reject
try:
    reject_button = WebDriverWait(driver, 5).until( EC.element_to_be_clickable((By.ID, "CybotCookiebotDialogBodyButtonDecline")) )
    reject_button.click()
    time.sleep(2)
except TimeoutException:
    print(f"No cookie dialog for {full_url}")
#extract year
year = re.search(r'\d{4}', driver.title).group()
print(f'Year {year}', f'Tile {driver.title}')

In [ ]:
for url in urls:
    full_url = url + part1
    driver.get(full_url)
    time.sleep(2)
    year = re.search(r'\d{4}', driver.title).group()
    label = 'THE_Asia_' + year
    print(f'Label {label} for Title {driver.title} and Year {year}')   

In [ ]:
#headers 
ranking_headers={}
for url in urls:
    full_url = url + part1
    driver.get(full_url)
    print(driver.title)
    time.sleep(2)
    #ranking_label = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "label[for='scores']")))
    #ranking_label.click()
    #time.sleep(2)
    
    table = driver.find_element(By.ID, "datatable-1")
    headers = table.find_elements(By.TAG_NAME, "th")
    ranking_headers[year] = [h.text.replace('\n', ' ').strip() for h in headers]
ranking_headers

In [ ]:
ranking_headers

In [ ]:
driver.refresh()